In [1]:
import sqlite3
import numpy as np
import pandas as pd
import config, data_load
from datatable import dt, f
from tqdm import tqdm

In [2]:
cache_length = 25
con = data_load.create_connection()
path_zip = config.PATH_DATABASE_ZIP
files = data_load.get_namelist_in_zip(path_zip)
sinasc_cols = pd.read_csv('data/check/sinasc_cols_count.csv')['variavel'].values

In [3]:
def adjust_df(df, sinasc_cols):
	df = df.to_pandas()
	for col in sinasc_cols:
		if col not in df.columns:
			df[col] = np.nan
	df = df.drop(['CONTADOR'], axis=1)
	return df

In [4]:
with (con):
	dfs = list()
	for f in tqdm(files):
		df = dt.fread(f'{path_zip}/{f}')
		df = adjust_df(df, sinasc_cols)
		dfs.append(df)
		if len(dfs) > cache_length:
			data_load.append_table_to_con(
				pd.concat(dfs),con)
			dfs = list()
	data_load.append_table_to_con(
			pd.concat(dfs),con)

 15%|█▌        | 103/675 [02:43<09:50,  1.03s/it] 

15% = `1.82GB`

In [ ]:
cnx = sqlite3.connect(config.PATH_DB)

In [ ]:
# df = pd.read_sql_query('select * from sinasc order by data_nasc, DTNASCMAE limit 5', cnx)
# df[sorted(df.columns)]

In [ ]:
sinasc_cols = pd.read_csv('data/check/sinasc_cols_count.csv')
sinasc_cols = sinasc_cols[sinasc_cols['percent'] >= 80]['variavel'].values
path_query = 'queries/check/sinasc.sql'
with open(path_query) as f:
	query_base = f.read()

In [ ]:
len(sinasc_cols)

In [ ]:
counts = list()
for var in tqdm(sinasc_cols):
	query = query_base.replace('*', var)
	records = data_load.df_query(query)['records'].values[0]
	counts.append({
		'variavel' : var,
		'recods' : records})
df_counts = pd.DataFrame(counts).sort_values(by='recods', ascending=False)
df = df_counts.sort_values(
	by=['recods', 'variavel'], ascending=[False, True]
)
df['percent'] = df['recods'] / 28672255 * 100

df.to_csv('data/check/sinasc_cols_count.csv', index=False)

In [ ]:
path_zip = config.PATH_DATABASE_ZIP
files = data_load.get_namelist_in_zip(path_zip)
pd.Series(files)

In [ ]:
df = dt.fread(f'{path_zip}/{files[2]}')
df

In [ ]:
d = list()
for f in tqdm(files):
	df = dt.fread(f'{path_zip}/{f}')
	rows, columns = df.shape
	d.append({'rows':rows, 'columns':columns})
df_shapes = pd.DataFrame(d)
df_shapes

In [ ]:
df_shapes.rows.sum()

In [ ]:
with (con := data_load.create_connection()):
	for f in tqdm(files[:10]):
		df = dt.fread(f'{path_zip}/{f}')
		df = df.to_pandas()
		df = df[sinasc_cols]
		# df.to_csv(f'{config.PATH_DATABASE}/csvs/{f}', index=False)

In [ ]:
with (con := data_load.create_connection()):
	dfs = list()
	for f in tqdm(files):
		df = dt.fread(f'{path_zip}/{f}')
		df = df.to_pandas()
		for col in sinasc_cols:
			if col not in df.columns:
				df[col] = np.nan
		df = df.drop(['CONTADOR'], axis=1)
		dfs.append(df)
		if len(dfs) > 50:
			df.to_sql(
				name = 'sinasc',
				con = con,
				if_exists = 'append', # append | replace
				index = False,
			)
			dfs = list()
	try:
		df.to_sql(
			name = 'sinasc',
			con = con,
			if_exists = 'append', # append | replace
			index = False,
		)
	except Exception as exc:
		print(exc)
		print(df.columns)